In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datetime import datetime

import os
import sys
cur_dir = os.path.dirname(os.path.abspath("__file__"))  # Gets the current notebook directory
src_dir = os.path.join(cur_dir, '../')  # Constructs the path to the 'src' directory
# Add the 'src' directory to sys.path
if src_dir not in sys.path:
    sys.path.append(src_dir)

from src.constant import *
from tqdm.notebook import tqdm

In [2]:
import torch.utils
import torch.utils.data


lookback = 30
future_steps = 40
dir = '../data/PandasData/Sampled/'

train_batch_size = 16
test_batch_size = 64


In [5]:
df1 = pd.read_csv("../data/PandasData/Sampled/PID003_NSL.csv")
df2 = pd.read_csv("../data/PandasData/Sampled/PID004_NSL.csv")

In [7]:
df1.columns

Index(['AGV_distance_X', 'AGV_distance_Y', 'AGV_speed_X', 'AGV_speed_Y',
       'AGV_speed', 'User_speed_X', 'User_speed_Y', 'User_speed',
       'User_velocity_X', 'User_velocity_Y', 'Wait_time', 'intent_to_cross',
       'Gazing_station', 'possible_interaction', 'facing_along_sidewalk',
       'facing_to_road', 'On_sidewalks', 'On_road', 'closest_station',
       'distance_to_closest_station', 'distance_to_closest_station_X',
       'distance_to_closest_station_Y', 'looking_at_AGV', 'start_station_X',
       'start_station_Y', 'end_station_X', 'end_station_Y',
       'distance_from_start_station_X', 'distance_from_start_station_Y',
       'distance_from_end_station_X', 'distance_from_end_station_Y',
       'facing_start_station', 'facing_end_station', 'GazeDirection_X',
       'GazeDirection_Y', 'GazeDirection_Z', 'AGV_X', 'AGV_Y', 'User_X',
       'User_Y', 'AGV_name', 'TimestampID', 'Timestamp',
       'looking_at_closest_station'],
      dtype='object')

In [24]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame
df = df1[df1['AGV_name'] == 'AGV1']
df = df.sort_values('Timestamp').reset_index(drop=True)

# Initialize the list to store frame dictionaries
frame_dicts = []

# Define the number of historical and future frames
num_his_frames = 30
num_fut_frames = 40

# Iterate over each frame (row) in the DataFrame
for i in range(len(df)):
    # Get current frame data
    row = df.iloc[i]

    # Compute 'ego2global_translation' (AGV position in global coordinates)
    ego2global_translation = np.array([row['AGV_X'], row['AGV_Y']])

    # Compute 'ego2global_rotation' (AGV heading angle)
    AGV_speed_X = row['AGV_speed_X']
    AGV_speed_Y = row['AGV_speed_Y']
    heading = np.arctan2(AGV_speed_Y, AGV_speed_X)
    ego2global_rotation = heading  # Alternatively, compute rotation matrix or quaternion

    # Collect 'ego_his_traj' (last 30 frames)
    start_idx = max(0, i - num_his_frames + 1)
    ego_his_traj_rows = df.iloc[start_idx:i + 1]
    ego_his_traj = ego_his_traj_rows[['AGV_X', 'AGV_Y']].to_numpy()

    # Pad historical trajectory if less than 30 frames
    if len(ego_his_traj) < num_his_frames:
        pad_size = num_his_frames - len(ego_his_traj)
        pad_values = np.tile(ego_his_traj[0], (pad_size, 1))
        ego_his_traj = np.vstack((pad_values, ego_his_traj))

    # Collect 'ego_fut_traj' (next 40 frames)
    end_idx = min(len(df), i + num_fut_frames)
    ego_fut_traj_rows = df.iloc[i + 1:end_idx + 1]
    ego_fut_traj = ego_fut_traj_rows[['AGV_X', 'AGV_Y']].to_numpy()

    # Pad future trajectory if less than 40 frames
    if len(ego_fut_traj) < num_fut_frames:
        pad_size = num_fut_frames - len(ego_fut_traj)
        if len(ego_fut_traj) > 0:
            pad_values = np.tile(ego_fut_traj[-1], (pad_size, 1))
        else:
            pad_values = np.tile(ego_his_traj[-1], (pad_size, 1))
        ego_fut_traj = np.vstack((ego_fut_traj, pad_values))

    # Adjust trajectories to the ego coordinate system
    cos_h = np.cos(-heading)
    sin_h = np.sin(-heading)
    rotation_matrix = np.array([[cos_h, -sin_h],
                                [sin_h, cos_h]])

    # Adjust historical trajectory
    ego_his_traj_adjusted = ego_his_traj - ego2global_translation
    ego_his_traj_ego = (rotation_matrix @ ego_his_traj_adjusted.T).T

    # Adjust future trajectory
    ego_fut_traj_adjusted = ego_fut_traj - ego2global_translation
    ego_fut_traj_ego = (rotation_matrix @ ego_fut_traj_adjusted.T).T

    # Prepare the frame dictionary
    frame_dict = {
        'ego2global_translation': ego2global_translation.tolist(),
        'ego2global_rotation': ego2global_rotation,  # Could be rotation matrix or quaternion
        'ego_his_traj': ego_his_traj_ego.tolist(),
        'ego_fut_traj': ego_fut_traj_ego.tolist(),
    }

    # Process User (agent) information
    users = []

    # Collect User's historical trajectory
    user_his_traj = ego_his_traj_rows[['User_X', 'User_Y']].to_numpy()
    if len(user_his_traj) < num_his_frames:
        pad_size = num_his_frames - len(user_his_traj)
        pad_values = np.tile(user_his_traj[0], (pad_size, 1))
        user_his_traj = np.vstack((pad_values, user_his_traj))

    user_his_traj_adjusted = user_his_traj - ego2global_translation
    user_his_traj_ego = (rotation_matrix @ user_his_traj_adjusted.T).T

    # Collect User's future trajectory
    user_fut_traj = ego_fut_traj_rows[['User_X', 'User_Y']].to_numpy()
    if len(user_fut_traj) < num_fut_frames:
        pad_size = num_fut_frames - len(user_fut_traj)
        if len(user_fut_traj) > 0:
            pad_values = np.tile(user_fut_traj[-1], (pad_size, 1))
        else:
            pad_values = np.tile(user_his_traj[-1], (pad_size, 1))
        user_fut_traj = np.vstack((user_fut_traj, pad_values))

    user_fut_traj_adjusted = user_fut_traj - ego2global_translation
    user_fut_traj_ego = (rotation_matrix @ user_fut_traj_adjusted.T).T

    # Compute User's yaw in ego coordinates

    # For current frame
    User_speed_X = row['User_speed_X']
    User_speed_Y = row['User_speed_Y']
    user_heading_global = np.arctan2(User_speed_Y, User_speed_X)
    user_heading_ego = user_heading_global - heading  # User's yaw in ego coordinates

    # For past frames
    user_his_speed_X = ego_his_traj_rows['User_speed_X'].to_numpy()
    user_his_speed_Y = ego_his_traj_rows['User_speed_Y'].to_numpy()
    user_his_heading_global = np.arctan2(user_his_speed_Y, user_his_speed_X)

    agv_his_speed_X = ego_his_traj_rows['AGV_speed_X'].to_numpy()
    agv_his_speed_Y = ego_his_traj_rows['AGV_speed_Y'].to_numpy()
    agv_his_heading = np.arctan2(agv_his_speed_Y, agv_his_speed_X)

    user_his_heading_ego = user_his_heading_global - agv_his_heading

    # Pad historical yaw if necessary
    if len(user_his_heading_ego) < num_his_frames:
        pad_size = num_his_frames - len(user_his_heading_ego)
        pad_values = np.full(pad_size, user_his_heading_ego[0])
        user_his_heading_ego = np.concatenate((pad_values, user_his_heading_ego))

    # For future frames
    user_fut_speed_X = ego_fut_traj_rows['User_speed_X'].to_numpy()
    user_fut_speed_Y = ego_fut_traj_rows['User_speed_Y'].to_numpy()
    user_fut_heading_global = np.arctan2(user_fut_speed_Y, user_fut_speed_X)

    # For future frames, adjust to current ego heading
    user_fut_heading_ego = user_fut_heading_global - heading  # Use current heading

    # Pad future yaw if necessary
    if len(user_fut_heading_ego) < num_fut_frames:
        pad_size = num_fut_frames - len(user_fut_heading_ego)
        if len(user_fut_heading_ego) > 0:
            pad_values = np.full(pad_size, user_fut_heading_ego[-1])
        else:
            pad_values = np.full(pad_size, user_his_heading_ego[-1])
        user_fut_heading_ego = np.concatenate((user_fut_heading_ego, pad_values))

    # Normalize angles to [-pi, pi]
    user_heading_ego = (user_heading_ego + np.pi) % (2 * np.pi) - np.pi
    user_his_heading_ego = (user_his_heading_ego + np.pi) % (2 * np.pi) - np.pi
    user_fut_heading_ego = (user_fut_heading_ego + np.pi) % (2 * np.pi) - np.pi

    user_info = {
        'user_current_yaw': user_heading_ego,
        'user_his_yaw': user_his_heading_ego.tolist(),
        'user_fut_yaw': user_fut_heading_ego.tolist(),
        'user_his_traj': user_his_traj_ego.tolist(),
        'user_fut_traj': user_fut_traj_ego.tolist(), 
        'user_fut_yaw_class': (user_fut_heading_ego // (np.pi / 4)).tolist()
        # Add more user features if needed
    }
    users.append(user_info)

    # Add user information to the frame dictionary
    frame_dict['users'] = users

    # Append the frame dictionary to the list
    frame_dicts.append(frame_dict)


In [25]:
frame_dicts[10]

{'ego2global_translation': [3700.02, 11357.516],
 'ego2global_rotation': 1.5707963267948966,
 'ego_his_traj': [[222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.38900000000103, 0.019999999999995428],
  [222.3890000000